In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[tensor([[ 0.8531, -0.1234],
         [ 0.7891, -0.1271],
         [ 1.0904,  0.3663],
         [ 0.3702,  1.1525],
         [-0.7900, -0.0496],
         [ 1.0894, -0.4549],
         [-0.0642, -1.4786],
         [ 1.0981,  0.7235],
         [-0.0791,  1.1520],
         [ 0.0930,  0.0213]]),
 tensor([[6.3238],
         [6.2005],
         [5.1274],
         [1.0243],
         [2.8028],
         [7.9162],
         [9.0891],
         [3.9226],
         [0.1300],
         [4.3247]])]

## 定义模型

In [4]:
from torch import nn   # nn是神经网络的缩写

net = nn.Sequential(nn.Linear(2,1))  # 2个输入1个输出

## 初始化模型参数

In [5]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数

In [6]:
loss = nn.MSELoss()

## 定义优化算法

In [10]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)
"""
小批量随机梯度下降算法是一种优化神经网络的标准工具， 
PyTorch在optim模块中实现了该算法的许多变种。
当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 
小批量随机梯度下降只需要设置lr值，这里设置为0.03。
"""

'\n小批量随机梯度下降算法是一种优化神经网络的标准工具， \nPyTorch在optim模块中实现了该算法的许多变种。\n当我们实例化一个SGD实例时，我们要指定优化的参数 （可通过net.parameters()从我们的模型中获得）以及优化算法所需的超参数字典。 \n小批量随机梯度下降只需要设置lr值，这里设置为0.03。\n'

## 训练

In [8]:
num_epochs = 3

for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()     # 每次小批量进行运算之前要进行梯度清零
        l.backward()            # 反向传播，在这里使用系统自带的损失函数，默认`reduction='mean'`,因此在这里不需要除以批量数了（参数更新的时候）
        trainer.step()          # 参数更新
    l = loss(net(features),labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000253
epoch 2, loss 0.000101
epoch 3, loss 0.000101


In [9]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0004, 0.0007])
b的估计误差： tensor([0.0010])
